# Extraction of models from DeciContas.br Dataset

In [ ]:
import os
import json

import pandas as pd
import numpy as np


from pprint import pprint
from langchain_openai import  AzureChatOpenAI, ChatOpenAI
from dotenv import load_dotenv
from sqlalchemy import (
    create_engine, Column, Integer, String, Float, Date, Boolean, Text, JSON
)
from sqlalchemy.orm import declarative_base, sessionmaker
from sqlalchemy.schema import DDL, CheckConstraint
from tools.prompt import generate_few_shot_ner_prompts
from tools.schema import (
    NERDecisao,
    Obrigacao
)

from datetime import timedelta


load_dotenv()

'''
gpt4turbo = AzureChatOpenAI(
    deployment_name="gpt-4-turbo",
    model_name="gpt-4",
)
'''

gpt4turbo = ChatOpenAI(
    model="gpt-4-turbo",
    temperature=0.0
)

extractor_gpt4turbo = gpt4turbo.with_structured_output(NERDecisao, include_raw=False, method="json_schema")
extractor_obrigacao_gpt4turbo = gpt4turbo.with_structured_output(
    Obrigacao, include_raw=False, method="json_schema")


def safe_int(value):
    if pd.isna(value):
        return None
    return int(value)



: 

In [ ]:
all_decisions = pd.read_csv("dataset/all_decisions.csv")
all_decisions['year'] = pd.to_datetime(all_decisions['datasessao']).dt.year
all_decisions_2023 = all_decisions[all_decisions['year'] >= 2023]

In [ ]:
len(all_decisions_2023), len(all_decisions) 

In [12]:
resultados = []
errors = []
for index, row in all_decisions_2023.iterrows():
    if index % 10 == 0 and index > 0:
        print(f"Processando instância {index} de {len(all_decisions_2023)}")
        all_decisions_2023.to_csv("dataset/all_decisions_2023_ner.csv", index=True)

    try:
        prompt_with_few_shot = generate_few_shot_ner_prompts(row['texto_acordao'])
        
        result = extractor_gpt4turbo.invoke(prompt_with_few_shot)
        all_decisions_2023.at[index, 'ner_decisao'] = result
        resultados.append(result)
    except Exception as e:
        errors.append({
            'index': index,
            'error': str(e),
            'row': row.to_dict()
        })

/tmp/ipykernel_12742/1431655093.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_decisions_2023.at[index, 'ner_decisao'] = result


Processando instância 690 de 8728
Processando instância 700 de 8728
Processando instância 710 de 8728
Processando instância 720 de 8728
Processando instância 730 de 8728
Processando instância 740 de 8728
Processando instância 750 de 8728
Processando instância 760 de 8728
Processando instância 770 de 8728
Processando instância 780 de 8728
Processando instância 790 de 8728
Processando instância 800 de 8728
Processando instância 810 de 8728
Processando instância 820 de 8728
Processando instância 1360 de 8728
Processando instância 1370 de 8728
Processando instância 1380 de 8728
Processando instância 1390 de 8728
Processando instância 1400 de 8728
Processando instância 1410 de 8728
Processando instância 1420 de 8728
Processando instância 1430 de 8728
Processando instância 1440 de 8728
Processando instância 1450 de 8728
Processando instância 1460 de 8728
Processando instância 1470 de 8728
Processando instância 1480 de 8728
Processando instância 1490 de 8728
Processando instância 1500 de 8728

In [14]:
for e in errors:
    prompt_with_few_shot = generate_few_shot_ner_prompts(e['row']['texto_acordao'])
    result = extractor_gpt4turbo.invoke(prompt_with_few_shot)
    all_decisions_2023.at[e['index'], 'ner_decisao'] = result
    resultados.append(result)

In [16]:
all_decisions_2023.to_pickle("dataset/all_decisions_2023_ner.pkl")

# Creating Obrigacao database in local sqlite

In [306]:
Base = declarative_base()
engine = create_engine(
    f"mssql+pymssql://{os.getenv("SQL_SERVER_USER")}:{os.getenv("SQL_SERVER_PASS")}@{os.getenv("SQL_SERVER_HOST")}/{os.getenv("SQL_SERVER_DB")}",
    echo=True
)
#engine = create_engine("sqlite:///dataset/sqlite/obrigacoes.db", echo=True)
SessionLocal = sessionmaker(bind=engine)

In [307]:
class ObrigacaoORM(Base):
    __tablename__ = "Obrigacao"

    IdObrigacao = Column(Integer, primary_key=True, index=True)
    IdProcesso = Column(Integer, nullable=False)
    IdComposicaoPauta = Column(Integer, nullable=False)
    IdVotoPauta = Column(Integer, nullable=False)
    DescricaoObrigacao = Column(Text, nullable=False)
    DeFazer = Column(Boolean, default=True)
    Prazo = Column(String, nullable=True)
    DataCumprimento = Column(Date, nullable=True)
    OrgaoResponsavel = Column(String, nullable=True)
    IdOrgaoResponsavel = Column(Integer, nullable=True)
    TemMultaCominatoria = Column(Boolean, default=False)
    NomeResponsavelMultaCominatoria = Column(String, nullable=True)
    DocumentoResponsavelMultaCominatoria = Column(String, nullable=True)
    IdPessoaMultaCominatoria = Column(Integer, nullable=True)
    ValorMultaCominatoria = Column(Float, nullable=True)
    PeriodoMultaCominatoria = Column(String, nullable=True)
    EMultaCominatoriaSolidaria = Column(Boolean, default=False)
    SolidariosMultaCominatoria = Column(JSON, nullable=True)

In [ ]:
class RecomendacaoORM(Base):
    __tablename__ = "Recomendacao"

    IdRecomendacao = Column(Integer, primary_key=True, index=True)
    IdProcesso = Column(Integer, nullable=False)
    IdComposicaoPauta = Column(Integer, nullable=False)
    IdVotoPauta = Column(Integer, nullable=False)
    DescricaoRecomendacao = Column(Text, nullable=True)
    PrazoCumprimentoRecomendacao = Column(String, nullable=True)
    DataCumprimentoRecomendacao = Column(Date, nullable=True)
    NomeResponsavel = Column(String, nullable=True)
    IdPessoaResponsavel = Column(Integer, nullable=True)
    OrgaoResponsavel = Column(String, nullable=True)
    IdOrgaoResponsavel = Column(Integer, nullable=True)

    def __repr__(self):
        return f"<Recomendacao(IdRecomendacao={self.IdRecomendacao}, descricao_recomendacao='{self.descricao_recomendacao[:30]}...')>"

In [309]:
#Base.metadata.drop_all(bind=engine)
Base.metadata.create_all(bind=engine)

2025-07-27 16:37:03,274 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-07-27 16:37:03,274 INFO sqlalchemy.engine.Engine SELECT [INFORMATION_SCHEMA].[TABLES].[TABLE_NAME] 
FROM [INFORMATION_SCHEMA].[TABLES] 
WHERE ([INFORMATION_SCHEMA].[TABLES].[TABLE_TYPE] = CAST(%(table_type_1)s AS NVARCHAR(max)) OR [INFORMATION_SCHEMA].[TABLES].[TABLE_TYPE] = CAST(%(table_type_2)s AS NVARCHAR(max))) AND [INFORMATION_SCHEMA].[TABLES].[TABLE_NAME] = CAST(%(table_name_1)s AS NVARCHAR(max)) AND [INFORMATION_SCHEMA].[TABLES].[TABLE_SCHEMA] = CAST(%(table_schema_1)s AS NVARCHAR(max))
2025-07-27 16:37:03,275 INFO sqlalchemy.engine.Engine [cached since 10.51s ago] {'table_type_1': 'BASE TABLE', 'table_type_2': 'VIEW', 'table_name_1': 'Obrigacao', 'table_schema_1': 'dbo'}
2025-07-27 16:37:03,313 INFO sqlalchemy.engine.Engine 
CREATE TABLE [Obrigacao] (
	[IdObrigacao] INTEGER NOT NULL IDENTITY, 
	[IdProcesso] INTEGER NOT NULL, 
	[IdComposicaoPauta] INTEGER NOT NULL, 
	[IdVotoPauta] INTEGER NOT NULL, 
	[De

# Extracting Obrigacoes using GPT4 Turbo

In [312]:
decisoes_json = json.load(open("dataset/decisoes_raw_2020_062025.json", "r", encoding="utf-8"))

In [313]:
def find_obrigacao_by_descricao(df_ob, descricao):
    return [i for i,r in df_ob.iterrows() if descricao in r['obrigacoes'][0].descricao_obrigacao][0]

def get_id_pessoa_multa_cominatoria(row, result_obrigacao):
    """
    Obtém o ID da pessoa responsável pela multa cominatória.
    """
    if result_obrigacao.documento_responsavel_multa_cominatoria:
        return [p['id_pessoa'] for p in row['responsaveis'] if p['documento_responsavel'] == result_obrigacao.documento_responsavel_multa_cominatoria][0]
    return None

def get_pessoas_str(pessoas):    
    pessoas_str = []
    for pessoa in pessoas:
        nome = pessoa.get('nome_responsavel', 'Desconhecido')
        documento = pessoa.get('documento_responsavel', 'Desconhecido')
        tipo = pessoa.get('tipo_responsavel', 'Desconhecido')
        if tipo == 'F':
            tipo = 'Física'
        elif tipo == 'J':
            tipo = 'Jurídica'
        pessoas_str.append(f"{nome} ({tipo} - {documento})")
    
    return ", ".join(pessoas_str)

def get_prompt_obrigacao(row, obrigacao):
    data_sessao = row['data_sessao']
    texto_acordao = row['texto_acordao']
    orgao_responsavel = row['orgao_responsavel']
    pessoas_responsaveis = row['responsaveis']


    return f"""
    Você é um Auditor de Controle Externo do TCE/RN. Sua tarefa é analisar o voto e extrair a obrigação imposta, preenchendo os campos do objeto Obrigacao.

    Data da Sessão: {data_sessao.strftime('%d/%m/%Y')}
    Obrigação detectada: {obrigacao.descricao_obrigacao}
    Texto do Acordão: {texto_acordao}
    Órgão Responsável: {orgao_responsavel}
    Pessoas Responsáveis: {get_pessoas_str(pessoas_responsaveis)}

    Dado esse contexto, preencha os campos da seguinte forma:
    - descricao_obrigacao: Descrição da obrigação imposta.
    - tipo: Tipo da obrigação (fazer/não fazer).
    - prazo: Prazo estipulado para cumprimento. Extraia o texto indicando o prazo, se houver. Exemplo: "90 dias".
    - data_cumprimento: Extraia do prazo do acórdão como data de início e faça o cálculo da data de cumprimento. Exemplo: 2025-09-13
    - orgao_responsavel: Órgão responsável pelo cumprimento da obrigação. Pessoa jurídica.
    - tem_multa_cominatoria: Indique se há multa cominatória associada à obrigação.
    - nome_responsavel_multa_cominatoria: Nome do responsável pela obrigação, se houver multa cominatória. Pessoa física responsável.
    - documento_responsavel_multa_cominatoria: Documento do responsável pela obrigação, se houver multa cominatória.
    - valor_multa_cominatoria: Se houver multa cominatória, preencha o valor.
    - periodo_multa_cominatoria: Período da multa cominatória, se houver.
    - e_multa_cominatoria_solidaria: Indique se a multa cominatória é solidária.
    - solidarios_multa_cominatoria: Lista de responsáveis solidários da multa cominatória.

    Use somente as informações do texto do acórdão e dos dados fornecidos. Não inclua informações adicionais ou suposições.
    Se o órgão responsável não estiver disponível, preencha o campo orgão_responsavel com "Desconhecido".
    """

def extract_obrigacao(row, obrigacao):
    prompt_obrigacao = get_prompt_obrigacao(row, obrigacao)
    return extractor_obrigacao_gpt4turbo.invoke(prompt_obrigacao)

def insert_obrigacao(db_session, obrigacao: Obrigacao, row):
    orm_obj = ObrigacaoORM(
        IdProcesso=safe_int(row['id_processo']),
        IdComposicaoPauta=safe_int(row['id_composicao_pauta']),
        IdVotoPauta=safe_int(row['id_voto_pauta']),
        DescricaoObrigacao=obrigacao.descricao_obrigacao,
        DeFazer=obrigacao.de_fazer,
        Prazo=obrigacao.prazo,
        DataCumprimento=obrigacao.data_cumprimento,
        OrgaoResponsavel=obrigacao.orgao_responsavel,
        IdOrgaoResponsavel=safe_int(row['id_orgao_responsavel']),
        TemMultaCominatoria=obrigacao.tem_multa_cominatoria,
        NomeResponsavelMultaCominatoria=obrigacao.nome_responsavel_multa_cominatoria,
        DocumentoResponsavelMultaCominatoria=obrigacao.documento_responsavel_multa_cominatoria,
        IdPessoaMultaCominatoria=get_id_pessoa_multa_cominatoria(row, obrigacao),
        ValorMultaCominatoria=obrigacao.valor_multa_cominatoria,
        PeriodoMultaCominatoria=obrigacao.periodo_multa_cominatoria,
        EMultaCominatoriaSolidaria=obrigacao.e_multa_cominatoria_solidaria,
        SolidariosMultaCominatoria=obrigacao.solidarios_multa_cominatoria
    )
    db_session.add(orm_obj)
    db_session.commit()
    return orm_obj

In [314]:
keys = [k for k in decisoes_json.keys()]
decisoes = decisoes_json[keys[0]]
df_decisoes_raw = pd.DataFrame(decisoes)

df_decisoes_raw.columns = [c.lower() for c in df_decisoes_raw.columns]

all_ner = pd.read_pickle("dataset/all_decisions_2023_ner.pkl")
all_ner['idvotopauta'] = all_ner['idvotopauta'].astype(str).str.replace('.0', '')
obrigacoes = [(d.idcomposicaopauta, d.idvoto, d.idvotopauta, d['ner_decisao'].obrigacoes) for _,d in all_ner.iterrows() if d['ner_decisao'].obrigacoes != []]
df_ob = pd.DataFrame(obrigacoes, columns=['id_composicao_pauta', 'id_voto', 'id_voto_pauta', 'obrigacoes'])
df_ob['id_voto_pauta'] = df_ob['id_voto_pauta'].astype('int64')


In [315]:
group_cols = [
        'id_processo', 'numero_processo', 'ano_processo', 'id_composicao_pauta', 
        'id_voto_pauta', 'numero_sessao', 'ano_sessao', 'data_sessao', 'relatorio',
        'fundamentacao_voto', 'conclusao', 'texto_acordao', 'orgao_responsavel', 'id_orgao_responsavel',
]

# Define as colunas para criar o dicionário de pessoas
person_cols = ['nome_responsavel', 'documento_responsavel', 'tipo_responsavel', 'id_pessoa']

# Agrupa o DataFrame e aplica uma função lambda para criar a lista de dicionários
df_decisoes = df_decisoes_raw.groupby(group_cols, dropna=False).apply(
    lambda x: pd.Series({'responsaveis': x[person_cols].apply(
        lambda y: y.dropna().to_dict(), axis=1
    ).tolist()})
).reset_index()


df_ob = df_ob[['id_composicao_pauta', 'id_voto_pauta', 'obrigacoes']].merge(df_decisoes, on=['id_composicao_pauta', 'id_voto_pauta'], how='left')
df_ob['data_sessao'] = pd.to_datetime(df_ob['data_sessao'], errors='coerce')
df_ob['n_obrigacoes'] = df_ob['obrigacoes'].apply(lambda x: len(x) if isinstance(x, list) else 0)


/var/folders/39/p0t9g2qn1gbcvnwqyz79d34c0000gn/T/ipykernel_1200/3350572719.py:11: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_decisoes = df_decisoes_raw.groupby(group_cols, dropna=False).apply(


In [317]:
errors = []
k = 0
for _, row in df_ob.iterrows():
    k += 1
    if k % 10 == 0 and k > 0:
        print(f"Processando instância {k} de {len(df_ob)}")

    for obrigacao in row['obrigacoes']:
        try:
            result_obrigacao = extract_obrigacao(row, obrigacao)
            db_session = SessionLocal()
            insert_obrigacao(db_session, result_obrigacao, row)
            db_session.close()
        except Exception as e:
            print(f"Erro ao processar obrigação: {e}")
            errors.append((row['id_processo'], obrigacao, str(e)))


2025-07-27 16:39:07,877 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-07-27 16:39:07,879 INFO sqlalchemy.engine.Engine INSERT INTO [Obrigacao] ([IdProcesso], [IdComposicaoPauta], [IdVotoPauta], [DescricaoObrigacao], [DeFazer], [Prazo], [DataCumprimento], [OrgaoResponsavel], [IdOrgaoResponsavel], [TemMultaCominatoria], [NomeResponsavelMultaCominatoria], [DocumentoResponsavelMultaCominatoria], [IdPessoaMultaCominatoria], [ValorMultaCominatoria], [PeriodoMultaCominatoria], [EMultaCominatoriaSolidaria], [SolidariosMultaCominatoria]) OUTPUT inserted.[IdObrigacao] VALUES (%(IdProcesso)s, %(IdComposicaoPauta)s, %(IdVotoPauta)s, %(DescricaoObrigacao)s, %(DeFazer)s, %(Prazo)s, %(DataCumprimento)s, %(OrgaoResponsavel)s, %(IdOrgaoResponsavel)s, %(TemMultaCominatoria)s, %(NomeResponsavelMultaCominatoria)s, %(DocumentoResponsavelMultaCominatoria)s, %(IdPessoaMultaCominatoria)s, %(ValorMultaCominatoria)s, %(PeriodoMultaCominatoria)s, %(EMultaCominatoriaSolidaria)s, %(SolidariosMultaCominatoria

In [318]:
len(errors)

67

In [319]:
errors

[(nan,
  NERObrigacao(descricao_obrigacao='remetido à Diretoria de Atos de Pessoal para que identifique a existência de processo de pensão correlato. Por fim, caso assim não seja possível, que seja notificado o Instituto de Previdência dos Servidores do Estado do Rio Grande do Norte para providenciar a remessa do processo de pensão ou comunicar a inexistência de dependente, para fins de arquivamento.'),
  'NaTType does not support strftime'),
 (nan,
  NERObrigacao(descricao_obrigacao='encaminhamento dos autos à Diretoria de Expediente – DE, para que seja retirado o caráter sigiloso dos presentes autos'),
  'NaTType does not support strftime'),
 (nan,
  NERObrigacao(descricao_obrigacao='devem os autos seguir para a Diretoria de Atos e Execuções – DAE para que seja realizada a cientificação do Representante, nos termos do art. 12, caput da Resolução n.º 16/2020-TCE'),
  'NaTType does not support strftime'),
 (nan,
  NERObrigacao(descricao_obrigacao='Determinar à SECEX, que elabore e divu